In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error,r2_score

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving updated - Copy.xlsx to updated - Copy.xlsx


In [ ]:
df = pd.read_excel("updated - Copy.xlsx")

In [ ]:
# Summary statistics for numerical columns (range, mean, etc.)
numerical_summary = df.describe().T
numerical_summary['Range'] = numerical_summary['max'] - numerical_summary['min']
numerical_summary = numerical_summary[['min', 'max', 'mean', 'std', 'Range']]
print("Numerical Summary:\n", numerical_summary)

# Unique levels for each column
unique_counts = df.nunique()
print("\nUnique Levels in Each Column:\n", unique_counts)

# Data types to identify categorical and numerical features
print("\nData Types:\n", df.dtypes)

# Distribution analysis for numerical features
for column in df.select_dtypes(include=['float64', 'int64']).columns:
    print(f"\n{column} Value Counts:\n", df[column].value_counts(bins=5))  # Binned count to view ranges

# Count of unique values for each categorical feature
categorical_columns = df.select_dtypes(include=['object']).columns
for column in categorical_columns:
    print(f"\n{column} Levels:\n", df[column].value_counts())

Numerical Summary:
                              min        max       mean        std      Range
Tc                      5.000000  25.000000  18.043478   7.822356  20.000000
inlet feed temprature  40.000000  90.000000  67.391304  17.460222  50.000000
feed salinity           2.000000  43.000000  19.826087  20.473768  41.000000
feed flow rate          2.500000   4.650000   4.434783   0.551289   2.150000
coolant flow rate       2.000000   3.650000   3.473913   0.483295   1.650000
flux                   10.832712  97.352766  50.241077  24.536088  86.520054

Unique Levels in Each Column:
 Tc                        5
inlet feed temprature     6
feed salinity             2
feed flow rate            4
coolant flow rate         3
flux                     69
dtype: int64

Data Types:
 Tc                         int64
inlet feed temprature      int64
feed salinity              int64
feed flow rate           float64
coolant flow rate        float64
flux                     float64
dtype: object

T

In [ ]:
for column in df.columns:
    print(f"Unique values in {column}:\n", df[column].unique(), "\n")

Unique values in Tc:
 [ 5 10 15 20 25] 

Unique values in inlet feed temprature:
 [40 50 60 70 80 90] 

Unique values in feed salinity:
 [ 2 43] 

Unique values in feed flow rate:
 [4.6  4.65 2.5  3.5 ] 

Unique values in coolant flow rate:
 [3.65 2.   2.9 ] 

Unique values in flux:
 [24.51602776 37.91709896 51.54711087 64.2671026  80.1735049  96.99299013
 18.87789873 32.83746913 47.95754124 66.32588634 81.90959324 92.85331436
 17.52177829 29.12520553 48.892151   61.10782146 76.3855342  89.82486484
 17.31242839 30.72202952 45.06099869 60.14320304 73.73867778 89.37876219
 11.47634218 26.13665263 36.67713193 48.34120155 66.37182054 75.226452
 23.78596988 36.97331147 50.9490085  65.97882809 79.69173997 97.35276577
 10.83271209 18.23057999 30.89123304 44.60489041 57.26517072 74.39988072
 16.91479052 28.78634263 46.44662293 60.68622335 76.24161324 90.21880125
 14.7555093  16.77725118 19.17223339 34.09172093 35.73459716 43.6272097
 53.99721546 60.         72.06322866 18.29896907 20.10309278 

In [ ]:

# Extract the target column
target_column = df.columns[-1]  # Get the name of the last column
flux_values = df[target_column]

# Calculate the minimum, maximum, and range values
min_flux = flux_values.min()
max_flux = flux_values.max()
range_flux = max_flux - min_flux


print("Minimum Flux:", min_flux)
print("Maximum Flux:", max_flux)
print("Range of Flux:", range_flux)


Minimum Flux: 10.8327120918443
Maximum Flux: 97.3527657671089
Range of Flux: 86.5200536752646


In [ ]:
# Step 1: Extract features and target from DataFrame
X = df.iloc[:, :-1].to_numpy()
y = df.iloc[:, -1].to_numpy()

# Verify the shapes of X and y
print("X.shape:", X.shape)
print("y.shape:", y.shape)

X.shape: (69, 5)
y.shape: (69,)


In [ ]:
# this is going to search for the best structure using cross validation

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size= 0.2 , random_state=42)
kf = KFold (n_splits= 5 , shuffle = True , random_state=42)
mse_list = []
r2_list = []
for train_indices,dev_indices in kf.split(X_train) :
  X_train_fold,X_dev_fold = X_train[train_indices],X_train[dev_indices]
  y_train_fold,y_dev_fold = y_train[train_indices],y_train[dev_indices]
  scaler = StandardScaler()
  X_train_fold_scaled = scaler.fit_transform(X_train_fold)
  X_dev_fold_scaled = scaler.transform(X_dev_fold)

  model = Sequential([Dense(units = 64 , activation = 'relu' , input_shape= (X_train_fold_scaled.shape[1], )),
                      Dense(units = 32 , activation = 'relu'),
                      Dense(units = 16 , activation = 'relu'),
                      Dense(units = 1 , activation = 'linear'),
                      ])
  model.compile(loss = 'mean_squared_error' , optimizer = tf.keras.optimizers.Adam(0.001))
  model.fit(X_train_fold_scaled,y_train_fold, epochs = 800 , verbose = 0)
  y_dev_fold_prediction = model.predict(X_dev_fold_scaled)
  mse = mean_squared_error(y_dev_fold,y_dev_fold_prediction)
  r2 = r2_score(y_dev_fold,y_dev_fold_prediction)
  mse_list.append(mse)
  r2_list.append(r2)
mse_avrage = np.mean(mse_list)
r2_avrage = np.mean(r2_list)
print(f" mse for all folds is {mse_list}, the avrage of them is {mse_avrage:.3f}")
print(f'and for r2 the list is {r2_list} and the average is {r2_avrage:.3f}')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
 mse for all folds is [15.011268822770017, 4.054454614308049, 15.323447088673525, 26.50286118608766, 5.697584806305143], the avrage of them is 13.318
and for r2 the list is [0.9617689887645825, 0.9946632982830148, 0.8842218939499483, 0.9689701414077484, 0.9898593891581382] and the average is 0.960


In [ ]:
from

In [ ]:
# Above, we have chosen the best structure of our neural network based on it's performance on the kfold or cross validation
# Here , we will train our model using the selected ANN though with all of the training set not the folds and evaluate on the test set

# Scaling the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Building the model with L2 regularization
model = Sequential([
    Dense(units=64, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)),
    Dense(units=32, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(units=16, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(units=1, activation='linear')
])

# Compiling the model
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.001))

# Training the model
model.fit(X_train_scaled, y_train, epochs=800, verbose=0)

# Evaluating the final model on the test set
y_train_prediction = model.predict(X_train_scaled)
mse_train = mean_squared_error(y_train, y_train_prediction)
r2_train = r2_score(y_train, y_train_prediction)

y_test_prediction = model.predict(X_test_scaled)
mse_test = mean_squared_error(y_test, y_test_prediction)
r2_test = r2_score(y_test, y_test_prediction)

# Printing the evaluation metrics
print(f'mse for the train is: {mse_train:.3f}')
print(f'R^2 for the train is: {r2_train:.2f}')
print(f'mse for the test is: {mse_test:.3f}')
print(f'R^2 for the test is: {r2_test:.3f}')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
mse for the train is: 2.071
R^2 for the train is: 1.00
mse for the test is: 6.246
R^2 for the test is: 0.992


In [ ]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE for the test dataset
mae_test = mean_absolute_error(y_test, y_test_prediction)

# Print the MAE result
print(f"mae for test dataset is: {mae_test:.3f}")


mae for test dataset is: 1.979


In [ ]:
from sklearn.metrics import mean_absolute_error
import numpy as np

# y_test: actual target values for the test set
# y_test_pred: predicted values for the test set
# mae_test: mean absolute error calculated previously

# Calculate MAE for the test dataset
mae_test = mean_absolute_error(y_test, y_test_prediction)

# Calculate the mean of the target values for the Relative MAE
mean_target = np.mean(y_test)

# Calculate Relative MAE for the test dataset
relative_mae_test = (mae_test / mean_target) * 100

# Print results
print(f"MAE for test dataset is: {mae_test:.3f}")
print(f"Relative MAE for test dataset is: {relative_mae_test:.2f}%")


MAE for test dataset is: 1.979
Relative MAE for test dataset is: 3.63%


In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd


# X and y have been verified as:
# X.shape: (69, 5)
# y.shape: (69,)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features (normalize)
scaler = StandardScaler()

# Define the parameter grid for grid search
param_grid = {
    'svr__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
    'svr__gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10, 'scale', 'auto'],
    'svr__epsilon': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    'pca__n_components': [2, 3, 4, 5]  # Adding PCA components as part of the grid search
}

# Create a pipeline with PCA and SVR
pipeline = Pipeline([
    ('scaler', scaler),
    ('pca', PCA()),
    ('svr', SVR(kernel='rbf'))
])

# Create the GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')

# Train the model
grid_search.fit(X_train, y_train)

# Get the best model with optimized parameters
best_model = grid_search.best_estimator_

# Make predictions on training and testing sets
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Evaluate model performance on training set
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# Evaluate model performance on testing set
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Print the results
print("Best Parameters:", grid_search.best_params_)
print("Train Mean Squared Error:", train_mse)
print("Train R2 Score:", train_r2)
print("Test Mean Squared Error:", test_mse)
print("Test R2 Score:", test_r2)


Best Parameters: {'pca__n_components': 4, 'svr__C': 10000, 'svr__epsilon': 1, 'svr__gamma': 0.01}
Train Mean Squared Error: 3.588724568034491
Train R2 Score: 0.9934715097449378
Test Mean Squared Error: 4.462107355995473
Test R2 Score: 0.9939855288687178


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [ ]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE for the test dataset
mae_test = mean_absolute_error(y_test, y_test_pred)

# Print the MAE result
print(f"mae for test dataset is: {mae_test:.3f}")


mae for test dataset is: 1.729


In [ ]:
from sklearn.metrics import mean_absolute_error
import numpy as np


# y_test: actual target values for the test set
# y_test_pred: predicted values for the test set
# mae_test: mean absolute error calculated previously

# Calculate MAE for the test dataset (if not already done)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Calculate the mean of the target values for the Relative MAE
mean_target = np.mean(y_test)

# Calculate Relative MAE for the test dataset
relative_mae_test = (mae_test / mean_target) * 100

# Print results
print(f"MAE for test dataset is: {mae_test:.3f}")
print(f"Relative MAE for test dataset is: {relative_mae_test:.2f}%")


MAE for test dataset is: 1.729
Relative MAE for test dataset is: 3.17%


In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV


# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizing features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Defining the XGBoost model with initial parameters
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'colsample_bytree': [0.7, 0.8, 0.9],
}

grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# **Identify the best model:**
best_model = grid_search.best_estimator_
print("Best model found through GridSearchCV:", best_model)

# Making predictions
y_train_pred = best_model.predict(X_train_scaled)
y_test_pred = best_model.predict(X_test_scaled)

# Evaluation using MSE and R-squared on training and testing sets
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Training Mean Squared Error (MSE):", train_mse)
print("Training R-squared (R^2):", train_r2)
print("Test Mean Squared Error (MSE):", test_mse)
print("Test R-squared (R^2):", test_r2)


Best model found through GridSearchCV: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)
Training Mean Squared Error (MSE): 0.4884531874135627
Training R-squared (R^2): 0.9991114219512728
Test Mean Squared Error (MSE): 4.2271599202018235
Test R-squared (R^2): 0.994302214339777


In [ ]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE for the test dataset
mae_test = mean_absolute_error(y_test, y_test_pred)

# Print the MAE result
print(f"mae for test dataset is: {mae_test:.3f}")


mae for test dataset is: 1.677


In [ ]:
from sklearn.metrics import mean_absolute_error
import numpy as np


# y_test: actual target values for the test set
# y_test_pred: predicted values for the test set
# mae_test: mean absolute error calculated previously

# Calculate MAE for the test dataset (if not already done)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Calculate the mean of the target values for the Relative MAE
mean_target = np.mean(y_test)

# Calculate Relative MAE for the test dataset
relative_mae_test = (mae_test / mean_target) * 100

# Print results
print(f"MAE for test dataset is: {mae_test:.3f}")
print(f"Relative MAE for test dataset is: {relative_mae_test:.2f}%")


MAE for test dataset is: 1.677
Relative MAE for test dataset is: 3.08%
